In [2]:
print("OK!")

OK!


In [58]:
import langchain
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
import pinecone 
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import Prompt
from langchain.llms import CTransformers  # From Quantized model usage
from tqdm import tqdm

In [2]:
import sentence_transformers, langchain, ctransformers
print(sentence_transformers.__version__)
print(langchain.__version__)


3.2.1
0.3.7


In [3]:
API_KEY = "90920abf-2654-4c44-95e5-a0a15b77d577"
# PINECONE_API_ENVIRONMENT = 

In [4]:
def load_pdf(data_dir):
    loader = DirectoryLoader(data_dir,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [6]:
docs = load_pdf("data")

In [7]:
#Create Chunks

def text_split(docs):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size= 500,
                                   chunk_overlap = 20)
    chunks = text_splitter.split_documents(documents=docs)
    return chunks



In [8]:
chunks = text_split(docs)
len(chunks), type(chunks)

(5860, list)

In [9]:
def download_huggingface_embedding():
    embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embedding

In [10]:
embedding_model = download_huggingface_embedding()

C:\Users\Mony\AppData\Local\Temp\ipykernel_14096\2763483091.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [11]:
len(embedding_model.embed_query("Hello World"))

384

In [12]:
'chunks'

'chunks'

In [13]:
emb = embedding_model.embed_query(chunks[0].page_content)
meta_data = chunks[0].page_content
print(emb)
print(meta_data)

[0.021459737792611122, -0.008097145706415176, -0.02617809921503067, 0.016104144975543022, -0.0319497250020504, 0.00957583636045456, 0.003219019388779998, 0.19288146495819092, -0.03242339938879013, -0.04132970795035362, 0.005839155521243811, 0.08297932893037796, 0.04542431980371475, 0.02660774439573288, -0.11355927586555481, 0.006353209260851145, -0.032044023275375366, -0.03040088526904583, -0.007295799441635609, -0.022904643788933754, -0.05075789615511894, 0.08314482122659683, 0.053239163011312485, 0.023431604728102684, -0.08658881485462189, 0.04784600809216499, -0.058837637305259705, -0.05568980798125267, -0.0017164155142381787, -0.016421087086200714, -0.0034193890169262886, 0.09850356727838516, 0.05514947697520256, -0.01397489383816719, -0.004718359559774399, -0.040588922798633575, 0.01893697679042816, 0.028466368094086647, -0.04568347707390785, 0.10384681075811386, 0.026701049879193306, -0.06810127943754196, -0.03478611633181572, -0.0038321022875607014, 0.0633513405919075, 0.0516095

In [14]:
print(emb)

[0.021459737792611122, -0.008097145706415176, -0.02617809921503067, 0.016104144975543022, -0.0319497250020504, 0.00957583636045456, 0.003219019388779998, 0.19288146495819092, -0.03242339938879013, -0.04132970795035362, 0.005839155521243811, 0.08297932893037796, 0.04542431980371475, 0.02660774439573288, -0.11355927586555481, 0.006353209260851145, -0.032044023275375366, -0.03040088526904583, -0.007295799441635609, -0.022904643788933754, -0.05075789615511894, 0.08314482122659683, 0.053239163011312485, 0.023431604728102684, -0.08658881485462189, 0.04784600809216499, -0.058837637305259705, -0.05568980798125267, -0.0017164155142381787, -0.016421087086200714, -0.0034193890169262886, 0.09850356727838516, 0.05514947697520256, -0.01397489383816719, -0.004718359559774399, -0.040588922798633575, 0.01893697679042816, 0.028466368094086647, -0.04568347707390785, 0.10384681075811386, 0.026701049879193306, -0.06810127943754196, -0.03478611633181572, -0.0038321022875607014, 0.0633513405919075, 0.0516095

In [63]:
from pinecone import Pinecone




pc = Pinecone(api_key=API_KEY)
index = pc.Index("medchatbot")

In [64]:
vector_store = PineconeVectorStore(index, embedding=embedding_model)

In [65]:
from uuid import uuid4

uuids = [str(uuid4()) for _ in chunks]

In [68]:
chunks[0]

Document(metadata={'source': 'data\\Gale Encyclopedia of Medicine. Vol. 1. 2nd ed.pdf', 'page': 1, 'text': 'The GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION'}, page_content='The GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION')

In [69]:
vector_store.add_documents(documents=chunks, id=uuids)

['51da49f1-110c-4aa7-83ad-a35c9b6062cb',
 'c005adcb-754b-4bf6-8659-dd624a30c5e0',
 '49439557-9698-40f2-8415-4f26e182bba1',
 '7730cac8-aea9-4d0e-9fa2-e1981625681b',
 '8ee8a449-1606-4afb-a58e-67de86111a5d',
 '9dd9be17-e36e-4cf2-9791-af9837092766',
 '0f9da545-790a-473f-a1c2-a6b1487f9873',
 '9431befe-1599-464c-bb6f-441d31695660',
 'a415a4ef-ddf7-4e04-a417-e2648055b5eb',
 'ebd69cd2-42ef-486e-884c-c25ffe55a14e',
 '5fa91c36-08c7-4ad0-8b0a-2ba52e464098',
 'f3dcd0ed-cead-4059-8072-c1945981c9ff',
 'b783093b-2795-4775-8c03-2c1c75e208bd',
 'ad8c3d9f-f5eb-4f0a-b1cf-34d27465fe7b',
 'ba699fd2-84f2-42b1-92ae-400d05db5a86',
 'bfb20e6e-7591-487c-ade7-7f8318cb44ca',
 '571e1337-8b44-4eb2-b0a1-243b84816cdc',
 '3b2e0db7-b9e7-46bb-bd23-cf76de3b5b5a',
 'c1005b70-f4e4-4040-b847-a7ce56d73d8f',
 'a5a4ddb1-76b2-43a9-847b-5eeffc58299f',
 'a14767f3-3cb9-4ce1-a3eb-edd2000cd18b',
 'eca380d4-31f9-4e5d-b589-8b2a891a0bc2',
 'b05acafa-3619-424f-b5fd-9cfc7f5eeb3e',
 '501287fa-2d9d-444f-9be4-05b32ae9d566',
 '60cdba19-3aad-

In [70]:
import time
time.sleep(10)  # Wait for the upserted vectors to be indexed

print(index.describe_index_stats())


{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 5860}},
 'total_vector_count': 5860}


In [71]:
query = " what is Abortion and  is types"

docs = vector_store.similarity_search(query=query, k=3)
docs

[Document(id='196be86e-871a-4314-8849-4a1fb0d62df1', metadata={'page': 24.0, 'source': 'data\\Gale Encyclopedia of Medicine. Vol. 1. 2nd ed.pdf'}, page_content='often involve more risk, more services, anesthesia, and\nsometimes a hospital stay. Insurance carriers and HMOs\nmay or may not cover the procedure. Federal law pro-\nGALE ENCYCLOPEDIA OF MEDICINE 2 11\nAbortion, therapeutic\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 11'),
 Document(id='785a538f-e845-4676-92c2-c41db82a1b28', metadata={'page': 22.0, 'source': 'data\\Gale Encyclopedia of Medicine. Vol. 1. 2nd ed.pdf'}, page_content='more fetuses.\nSelective reduction—Typically referred to in cases\nof multifetal pregnancy, when one or more fetuses\nare aborted to preserve the viability of the remain-\ning fetuses and decrease health risks to the mother.\nrate of complications. Abortions after 24 weeks are\nextremely rare and are usually limited to situations where\nthe life of the mother is in danger.\nPrecautions\nMost wome

In [44]:
prompt_templete = """
Use the FOllowing peice of information to answer the question.
if you don't know just say you don't you, don't ty to make up answer.

Context = {context}
Question= {question}

Only return helpful answer below and nothing else
Helpful answer"""

In [45]:
Prompt = PromptTemplate(
    template = prompt_templete,
    input_variables = ["context",  "question"])

chain_type_kwargs={"prompt":Prompt}


In [47]:
llm = CTransformers(model = "model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                    model_type = "llama",
                    config = {"max_new_tokens" : 512,
                              "temperature" : 0.8})

In [77]:
qa = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff",
    retriever = vector_store.as_retriever(search_kwargs = {"k" : 2}),
    return_source_documents = True,
    chain_type_kwargs = chain_type_kwargs)



In [78]:

result = qa.invoke({"query":query})
print("Response : ", result["result"])

KeyboardInterrupt: 